<a href="https://colab.research.google.com/github/SaiArja/Trees_Near_Electric_wires/blob/main/Electrical_lines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Required Libraries**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image

Reading the Labels file as a Data Frame

In [4]:
data_label = pd.read_csv('/content/drive/MyDrive/DataSets/DeepLearning/detect_trees_near_electric_wires/train.csv')

Seeing the First 5 rows of the dataFrame

In [5]:
data_label.head()

,file_name,Intersection
0,Image_1.jpg,0
1,Image_2.jpg,1
2,Image_3.jpg,1
3,Image_4.jpg,1
4,Image_5.jpg,1


***Observation:- ***

the Data Frame has two columns 

1.**file_name** which has file name

2.**Intersection** which is a binary value which tells any contact between the electric wire and the tree(1 - contact; 0 - No Contact)

In [8]:
data_label['Intersection'].value_counts()

1    826
0    616
Name: Intersection, dtype: int64

In the Whole data we have around 826 images which are showing the trees contacting the power lines and 618 images which show that power line are not contacting the trees

The data is kind of balanced now. we will see once again after splitting as train, validation, test

In [11]:
data_label.shape

(1442, 2)

In [49]:
data_label.Intersection = pd.Series(np.where(data_label.Intersection.values == 1, 'yes', 'no'),
          data_label.index)

In [50]:
from sklearn.model_selection import train_test_split

def split_stratified_into_train_val_test(df_input, stratify_colname='Intersection',
                                         frac_train=0.7, frac_val=0.15, frac_test=0.15,
                                         random_state=32):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test

In [51]:
df_train, df_val, df_test = split_stratified_into_train_val_test(data_label)

In [52]:
df_train.head()

,file_name,Intersection
788,Image_789.jpg,no
1255,Image_1256.jpg,yes
697,Image_698.jpg,yes
685,Image_686.jpg,no
1065,Image_1066.jpg,yes


In [53]:
df_train['Intersection'].value_counts()

yes    578
no     431
Name: Intersection, dtype: int64

In [54]:
df_val['Intersection'].value_counts()

yes    124
no      92
Name: Intersection, dtype: int64

In [55]:
df_test['Intersection'].value_counts()

yes    124
no      93
Name: Intersection, dtype: int64

In [56]:
train_folder = '/content/drive/MyDrive/DataSets/DeepLearning/detect_trees_near_electric_wires/train'

In [57]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data generator
train_gen = ImageDataGenerator(
rotation_range=45,
rescale=1./255,
horizontal_flip=True
)
val_gen = ImageDataGenerator(
rotation_range=45,
rescale=1./255,
horizontal_flip=True
)
test_gen = ImageDataGenerator(rescale = 1./255)

In [68]:
train_data = train_gen.flow_from_dataframe(dataframe = df_train, x_col='file_name',
                                          directory = train_folder,y_col = 'Intersection',class_mode="binary",
                                          batch_size = 32, shuffle = True, 
                                          target_size = (299, 299))

val_data = train_gen.flow_from_dataframe(dataframe = df_val, 
                                          directory = train_folder, x_col = 'file_name', 
                                          y_col = 'Intersection',
                                          batch_size = 32, shuffle = True, 
                                          class_mode="binary",target_size = (299, 299))


test_data = test_gen.flow_from_dataframe(dataframe = df_test, 
                                        directory = train_folder, x_col = 'file_name', 
                                        y_col = 'Intersection',
                                        batch_size = 32, shuffle = False, 
                                        target_size = (299, 299))

Found 1009 validated image filenames belonging to 2 classes.
Found 216 validated image filenames belonging to 2 classes.
Found 217 validated image filenames belonging to 2 classes.
